In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import expm
from scipy import integrate

from control_theory import utils

In [ ]:
A = np.array([
    [0, 1],
    [1, 0]
])

b = np.array([
    [0],
    [1]
])

In [ ]:
h = 0.1

A_d = expm(A * h)
b_d = A_d @ b

In [ ]:
print(A_d, b_d, sep='\n\n')

In [ ]:
C = np.column_stack([
    b_d,
    A_d @ b_d,
])

print(C)
print(C.shape)

In [ ]:
print(np.linalg.matrix_rank(C))
print(np.linalg.eigvals(C))

In [ ]:
theta = (
    -np.array([[0, 1]])
    @ np.linalg.inv(C)
    @ utils.create_char_pol(-0.08, 0.8)(A_d)
)

print(theta)

In [ ]:
def system(t, x, A: np.ndarray) -> np.ndarray:
    return A @ x

# Кусочно постоянное управление

In [ ]:
A = np.array([
    [0, 1],
    [1, 0]
])

b = np.array([
    [0],
    [1]
])

In [ ]:
h = 0.1
delta = 0.007

A_d = expm(A * h)

In [ ]:
def f_c(x):
    return expm(A * x) @ b

In [ ]:
b_d = integrate.quad_vec(f_c, 0, delta)[0]

print(b_d)

In [ ]:
C = np.column_stack([
    b_d,
    A_d @ b_d,
])

print(C)
print(C.shape)

In [ ]:
print(np.linalg.matrix_rank(C))
print(np.linalg.eigvals(C))

In [ ]:
theta = (
    -np.array([[0, 1]])
    @ np.linalg.inv(C)
    @ utils.create_char_pol(-0.2, 0.9)(A_d)
)

print(theta)

In [ ]:
def system(
        t: np.ndarray,
        x: np.ndarray,
        A: np.ndarray,
        b: np.ndarray,
        theta: np.ndarray,
        control_enabled: bool,
        x_0 = None,
) -> np.ndarray | int:
    if x_0 is None:
        x_0 = x

    if control_enabled:
        return A @ x + b @ theta @ x_0

    return A @ x

In [ ]:
start, stop = 0, 5
t = start
eps = 0.1 * delta

y_0 = np.array([0, 0.5])

tmp_le_h = delta

ts = [0]
ys = [np.array(y_0)]

control_enabled = True
while t < stop:
    if h - eps <= tmp_le_h:
        tmp_le_h -= h
        control_enabled = True

    sol = integrate.solve_ivp(
        system,
        (t, t + delta), y_0,
        args=(A, b, theta, control_enabled),
        method="RK45",
        t_eval=[t + delta]
    )

    control_enabled = False
    t += delta
    tmp_le_h += delta

    ts.append(t)
    ys.append(sol.y.ravel())
    y_0 = sol.y.ravel()

ys = np.asarray(ys)

In [ ]:
fig = plt.plot(ys[:, 0], ys[:, 1], alpha=1)